In [ ]:
import  pandas as pd


dfs = pd.read_html("https://fbref.com/en/squads/206d90db/Barcelona-Stats")
print(dfs)


[       Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                   Player             Nation                Pos   
0                   Pedri             es ESP                 MF   
1                Raphinha             br BRA              FW,MF   
2      Robert Lewandowski             pl POL                 FW   
3            Lamine Yamal             es ESP                 FW   
4             Pau Cubarsí             es ESP                 DF   
5            Jules Koundé             fr FRA                 DF   
6          Iñigo Martínez             es ESP                 DF   
7         Alejandro Balde             es ESP                 DF   
8             Marc Casado             es ESP                 MF   
9              Iñaki Peña             es ESP                 GK   
10      Wojciech Szczęsny             pl POL                 GK   
11            Eric García             es ESP              DF,MF   
12                   Gavi             es ESP              MF,

# We look at the columns of each table

In [19]:
for i, df in enumerate(dfs):
    print(f"🔢 Tabla {i} - Shape: {df.shape}")
    print(f"🧩 Columnas: {list(df.columns)}\n")

🔢 Tabla 0 - Shape: (41, 34)
🧩 Columnas: [('Unnamed: 0_level_0', 'Player'), ('Unnamed: 1_level_0', 'Nation'), ('Unnamed: 2_level_0', 'Pos'), ('Unnamed: 3_level_0', 'Age'), ('Unnamed: 4_level_0', 'MP'), ('Playing Time', 'Starts'), ('Playing Time', 'Min'), ('Playing Time', '90s'), ('Performance', 'Gls'), ('Performance', 'Ast'), ('Performance', 'G+A'), ('Performance', 'G-PK'), ('Performance', 'PK'), ('Performance', 'PKatt'), ('Performance', 'CrdY'), ('Performance', 'CrdR'), ('Expected', 'xG'), ('Expected', 'npxG'), ('Expected', 'xAG'), ('Expected', 'npxG+xAG'), ('Progression', 'PrgC'), ('Progression', 'PrgP'), ('Progression', 'PrgR'), ('Per 90 Minutes', 'Gls'), ('Per 90 Minutes', 'Ast'), ('Per 90 Minutes', 'G+A'), ('Per 90 Minutes', 'G-PK'), ('Per 90 Minutes', 'G+A-PK'), ('Per 90 Minutes', 'xG'), ('Per 90 Minutes', 'xAG'), ('Per 90 Minutes', 'xG+xAG'), ('Per 90 Minutes', 'npxG'), ('Per 90 Minutes', 'npxG+xAG'), ('Unnamed: 33_level_0', 'Matches')]

🔢 Tabla 1 - Shape: (60, 20)
🧩 Columnas: ['

# Extract the tables that are useful to us

In [43]:
TABLAS_UTILES = [0, 2, 3, 4, 5, 7, 8, 9, 10, 11]
URL = "https://fbref.com/en/squads/206d90db/Barcelona-Stats"

def extraer_tablas_utiles(URL: str, indices_utiles: list[int]) -> list[pd.DataFrame]:
    tablas = pd.read_html(URL, header=[0, 1])
    print(f"📊 Tablas totales encontradas: {len(tablas)}")
    
    tablas_utiles = []
    for i in indices_utiles:
        df = tablas[i].copy()
        print(f"\n✅ Tabla {i} seleccionada")
        print(f"Shape: {df.shape}")
        print(f"Columnas: {df.columns.tolist()[:5]} ...")
        tablas_utiles.append(df)
    
    return tablas_utiles

tablas = extraer_tablas_utiles(URL, TABLAS_UTILES)

for i, t in enumerate(tablas):
    print(f"✅ tabla[{i}] tipo: {type(t)}")


📊 Tablas totales encontradas: 14

✅ Tabla 0 seleccionada
Shape: (41, 34)
Columnas: [('Unnamed: 0_level_0', 'Player'), ('Unnamed: 1_level_0', 'Nation'), ('Unnamed: 2_level_0', 'Pos'), ('Unnamed: 3_level_0', 'Age'), ('Playing Time', 'MP')] ...

✅ Tabla 2 seleccionada
Shape: (5, 24)
Columnas: [('Unnamed: 0_level_0', 'Player'), ('Unnamed: 1_level_0', 'Nation'), ('Unnamed: 2_level_0', 'Pos'), ('Unnamed: 3_level_0', 'Age'), ('Playing Time', 'MP')] ...

✅ Tabla 3 seleccionada
Shape: (5, 31)
Columnas: [('Unnamed: 0_level_0', 'Player'), ('Unnamed: 1_level_0', 'Nation'), ('Unnamed: 2_level_0', 'Pos'), ('Unnamed: 3_level_0', 'Age'), ('Unnamed: 4_level_0', '90s')] ...

✅ Tabla 4 seleccionada
Shape: (30, 23)
Columnas: [('Unnamed: 0_level_0', 'Player'), ('Unnamed: 1_level_0', 'Nation'), ('Unnamed: 2_level_0', 'Pos'), ('Unnamed: 3_level_0', 'Age'), ('Unnamed: 4_level_0', '90s')] ...

✅ Tabla 5 seleccionada
Shape: (30, 29)
Columnas: [('Unnamed: 0_level_0', 'Player'), ('Unnamed: 1_level_0', 'Nation'), 

# Merge into one dataframe with an outer product

In [50]:
from functools import reduce
import pandas as pd
import re

def flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    def limpiar_col(col):
        if isinstance(col, tuple):
            col = "_".join(col)
        # Elimina prefijos tipo "Unnamed: d_level_d_"
        col = re.sub(r"^Unnamed: \d+_level_\d+_", "", col) 
        return col.strip()
    
    df.columns = [limpiar_col(col) for col in df.columns]
    return df


def limpiar_tabla(df: pd.DataFrame) -> pd.DataFrame:
    df = flatten_columns(df)

    # Detectar columna que contiene 'Player'
    col_player = next((col for col in df.columns if 'Player' in col), None)
    if col_player is None:
        print("⚠️ No se encontró columna de jugador.")
        return None

    # Eliminar filas de cabecera o totales
    df = df[~df[col_player].isin(['Player'])]
    df = df.dropna(subset=[col_player])
    df = df.rename(columns={col_player: 'Player'})

    # Eliminar columna Matches si existe
    if 'Matches' in df.columns:
        df.drop(columns='Matches', inplace=True)

    return df


def merge_controlado_por_player(tablas: list[pd.DataFrame]) -> pd.DataFrame:
    tablas_limpias = [limpiar_tabla(df) for df in tablas if df is not None]
    df_base = tablas_limpias[0]

    for i, df_nueva in enumerate(tablas_limpias[1:], start=1):
        columnas_base = set(df_base.columns)
        columnas_nueva = set(df_nueva.columns)

        columnas_duplicadas = (columnas_base & columnas_nueva) - {'Player'}
        if columnas_duplicadas:
            print(f"🔁 Paso {i}: eliminando duplicadas → {columnas_duplicadas}")
            df_nueva = df_nueva.drop(columns=columnas_duplicadas, errors='ignore')

        columnas_nuevas = set(df_nueva.columns) - columnas_base
        print(f"➕ Paso {i}: nuevas columnas añadidas → {columnas_nuevas}")

        df_base = pd.merge(df_base, df_nueva, on='Player', how='outer')

    return df_base

df_final = merge_controlado_por_player(tablas)
print(f"✅ DataFrame final: {df_final.shape[0]} jugadores y {df_final.shape[1]} columnas")




🔁 Paso 1: eliminando duplicadas → {'Playing Time_90s', 'Nation', 'Age', 'Pos', 'Playing Time_MP', 'Playing Time_Min', 'Playing Time_Starts'}
➕ Paso 1: nuevas columnas añadidas → {'Performance_GA90', 'Performance_D', 'Performance_SoTA', 'Penalty Kicks_Save%', 'Penalty Kicks_PKsv', 'Performance_GA', 'Performance_L', 'Performance_Save%', 'Performance_CS%', 'Penalty Kicks_PKatt', 'Performance_CS', 'Performance_W', 'Penalty Kicks_PKA', 'Penalty Kicks_PKm', 'Performance_Saves'}
🔁 Paso 2: eliminando duplicadas → {'Nation', 'Age', 'Pos'}
➕ Paso 2: nuevas columnas añadidas → {'Launched_Cmp%', 'Goal Kicks_Att', 'Crosses_Opp', 'Goal Kicks_Launch%', 'Goals_FK', 'Crosses_Stp%', 'Expected_/90', 'Expected_PSxG', 'Goals_OG', 'Goals_CK', 'Passes_Thr', 'Sweeper_#OPA', 'Goal Kicks_AvgLen', 'Passes_Launch%', 'Sweeper_AvgDist', 'Passes_AvgLen', 'Crosses_Stp', '90s', 'Launched_Cmp', 'Expected_PSxG+/-', 'Launched_Att', 'Goals_PKA', 'Goals_GA', 'Passes_Att (GK)', 'Expected_PSxG/SoT', 'Sweeper_#OPA/90'}
🔁 Paso

# Column cleaning

Index(['Player', 'Unnamed: 1_level_0_Nation', 'Unnamed: 2_level_0_Pos',
       'Unnamed: 3_level_0_Age', 'Playing Time_MP', 'Playing Time_Starts',
       'Playing Time_Min', 'Playing Time_90s', 'Performance_Gls',
       'Performance_Ast',
       ...
       'Performance_Crs', 'Performance_Int', 'Performance_TklW',
       'Performance_PKwon', 'Performance_PKcon', 'Performance_OG',
       'Performance_Recov', 'Aerial Duels_Won', 'Aerial Duels_Lost',
       'Aerial Duels_Won%'],
      dtype='object', length=207)

# Extrapolar a todos los equipos 

In [53]:
import requests
from bs4 import BeautifulSoup
import re

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; CrOS x86_64 14541.0.0)"
        "AppleWebKit/537.36 (KHTML, like Gecko)" 
        "Chrome/134.0.0.0 Safari/537.36 "
    )
}

BASE_URL = "https://fbref.com"
URL_LALIGA_STATS = "https://fbref.com/en/comps/12/La-Liga-Stats"

def extraer_urls_equipos(url_base: str) -> dict:
    resp = requests.get(url_base, headers=HEADERS)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.content, "html.parser")
    links = soup.select("a[href]")

    equipos_urls = {}
    for link in links:
        href = link.get("href", "")
        if re.match(r"^/en/squads/[a-z0-9]{8}/.*-Stats$", href):
            nombre_equipo = link.text.strip()
            url_completa = BASE_URL + href
            equipos_urls[nombre_equipo] = url_completa

    print(f"✅ Se encontraron {len(equipos_urls)} equipos")
    return equipos_urls

urls = extraer_urls_equipos(URL_LALIGA_STATS)
for nombre, url in urls.items():
    print(f"{nombre}: {url}")




HTTPError: 403 Client Error: Forbidden for url: https://fbref.com/en/comps/12/La-Liga-Stats

In [ ]:
<table class="stats_table sortable min_width force_mobilize now_sortable sticky_table eq2 re2 le2" id="results2024-2025121_overall" data-cols-to-freeze=",2"> <caption>La Liga Table</caption> <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup> <thead> <tr> <th aria-label="Rank" data-stat="rank" scope="col" class=" poptip sort_default_asc center" data-tip="&lt;strong&gt;Rank&lt;/strong&gt;&lt;br&gt;Squad finish in competition&lt;br&gt;Finish within the league or competition.&lt;br&gt;For knockout competitions may show final round reached.&lt;br&gt;Colors and arrows represent promotion/relegation or qualifiation for continental cups.&lt;br&gt;Trophy indicates team won league whether by playoffs or by leading the table.&lt;br&gt;Star indicates topped table in league USING another means of naming champion.">Rk</th> <th aria-label="Squad" data-stat="team" scope="col" class=" poptip sort_default_asc center">Squad</th> <th aria-label="Matches Played" data-stat="games" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Matches Played&lt;/strong&gt;&lt;br&gt;Matches Played by the player or squad">MP</th> <th aria-label="Wins" data-stat="wins" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Wins&lt;/strong&gt;&lt;br&gt;Wins">W</th> <th aria-label="Draws" data-stat="ties" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Draws&lt;/strong&gt;&lt;br&gt;Draws">D</th> <th aria-label="Losses" data-stat="losses" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Losses&lt;/strong&gt;&lt;br&gt;Losses">L</th> <th aria-label="Goals For" data-stat="goals_for" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Goals For&lt;/strong&gt;&lt;br&gt;Goals For">GF</th> <th aria-label="Goals Against" data-stat="goals_against" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Goals Against&lt;/strong&gt;&lt;br&gt;Goals Against">GA</th> <th aria-label="Goal Difference" data-stat="goal_diff" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Goal Difference&lt;/strong&gt;&lt;br&gt;Goal Difference">GD</th> <th aria-label="Points" data-stat="points" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Points&lt;/strong&gt;&lt;br&gt;Most leagues are ordered by points.&lt;br&gt;Three for a win and one for a draw.">Pts</th> <th aria-label="Points/Match" data-stat="points_avg" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Points/Match&lt;/strong&gt;&lt;br&gt;Points per Match Played">Pts/MP</th> <th aria-label="xG" data-stat="xg_for" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Expected Goals&lt;/strong&gt;&lt;br&gt;xG totals include penalty kicks, but do not include penalty shootouts (unless otherwise noted).&lt;br&gt;Provided by Opta.&lt;br&gt;An underline indicates there is a match that is missing data, but will be updated when available." data-filter="1" data-name="xG">xG</th> <th aria-label="xG Allowed" data-stat="xg_against" scope="col" class=" poptip sort_default_asc center" data-tip="&lt;strong&gt;xG Allowed&lt;/strong&gt;&lt;br&gt;&lt;strong&gt;Expected Goals Allowed&lt;/strong&gt;&lt;br&gt;xG totals include penalty kicks, but do not include penalty shootouts (unless otherwise noted).&lt;br&gt;Provided by Opta.&lt;br&gt;An underline indicates there is a match that is missing data, but will be updated when available.">xGA</th> <th aria-label="xG Difference" data-stat="xg_diff" scope="col" class=" poptip center" data-tip="&lt;strong&gt;xG Difference&lt;/strong&gt;&lt;br&gt;&lt;strong&gt;Expected Goals Difference&lt;/strong&gt;&lt;br&gt;xG totals include penalty kicks, but do not include penalty shootouts (unless otherwise noted).&lt;br&gt;Provided by Opta.&lt;br&gt;An underline indicates there is a match that is missing data, but will be updated when available." data-filter="1" data-name="xG Difference">xGD</th> <th aria-label="xG Difference/90" data-stat="xg_diff_per90" scope="col" class=" poptip hide_non_quals center" data-tip="&lt;strong&gt;xG Difference/90&lt;/strong&gt;&lt;br&gt;&lt;strong&gt;Expected Goals Difference per 90 Minutes&lt;/strong&gt;&lt;br&gt;xG totals include penalty kicks, but do not include penalty shootouts (unless otherwise noted).&lt;br&gt;Provided by Opta.&lt;br&gt;An underline indicates there is a match that is missing data, but will be updated when available.">xGD/90</th> <th aria-label="Attendance/Game" data-stat="attendance_per_g" scope="col" class=" poptip center" data-tip="&lt;strong&gt;Attendance/Game&lt;/strong&gt;&lt;br&gt;Attendance per game during this season, only for home matches">Attendance</th> <th aria-label="Top Team Scorer" data-stat="top_team_scorers" scope="col" class=" poptip center" data-tip="Only includes league matches for that season">Top Team Scorer</th> <th aria-label="Goalkeeper" data-stat="top_keeper" scope="col" class=" poptip center" data-tip="Goalkeeper with the most minutes in league play">Goalkeeper</th> <th aria-label="Notes" data-stat="notes" scope="col" class=" poptip center">Notes</th> </tr> </thead> <tbody> <tr data-row="0"><th scope="row" class="right qualifier qualification_indicator1" data-stat="rank" csk="1">1</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.206d90db.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/206d90db/Barcelona-Stats">Barcelona</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">28</td><td class="right " data-stat="ties">4</td><td class="right " data-stat="losses">6</td><td class="right " data-stat="goals_for">102</td><td class="right " data-stat="goals_against">39</td><td class="right " data-stat="goal_diff">+63</td><td class="right " data-stat="points">88</td><td class="right " data-stat="points_avg">2.32</td><td class="right " data-stat="xg_for">91.5</td><td class="right " data-stat="xg_against">41.9</td><td class="right " data-stat="xg_diff">+49.5</td><td class="right " data-stat="xg_diff_per90">+1.30</td><td class="right " data-stat="attendance_per_g" csk="45953">45,953</td><td class="right " data-stat="top_team_scorers" csk="27"><a href="/en/players/8d78e732/Robert-Lewandowski">Robert Lewandowski</a> - <span>27</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/cc1c5035/Inaki-Pena">Iñaki Peña </a></td><td class="left " data-stat="notes">→ Champions League via league finish</td></tr> <tr data-row="1"><th scope="row" class="right qualifier qualification_indicator1" data-stat="rank" csk="2">2</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.53a2f082.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/53a2f082/2024-2025/Real-Madrid-Stats">Real Madrid</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">26</td><td class="right " data-stat="ties">6</td><td class="right " data-stat="losses">6</td><td class="right " data-stat="goals_for">78</td><td class="right " data-stat="goals_against">38</td><td class="right " data-stat="goal_diff">+40</td><td class="right " data-stat="points">84</td><td class="right " data-stat="points_avg">2.21</td><td class="right " data-stat="xg_for">75.3</td><td class="right " data-stat="xg_against">42.8</td><td class="right " data-stat="xg_diff">+32.5</td><td class="right " data-stat="xg_diff_per90">+0.86</td><td class="right " data-stat="attendance_per_g" csk="69807">69,807</td><td class="right " data-stat="top_team_scorers" csk="31"><a href="/en/players/42fd9c7f/Kylian-Mbappe">Kylian Mbappé</a> - <span>31</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/1840e36d/Thibaut-Courtois">Thibaut Courtois</a></td><td class="left " data-stat="notes">→ Champions League via league finish</td></tr> <tr data-row="2"><th scope="row" class="right qualifier qualification_indicator1" data-stat="rank" csk="3">3</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.db3b9613.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/db3b9613/2024-2025/Atletico-Madrid-Stats">Atlético Madrid</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">22</td><td class="right " data-stat="ties">10</td><td class="right " data-stat="losses">6</td><td class="right " data-stat="goals_for">68</td><td class="right " data-stat="goals_against">30</td><td class="right " data-stat="goal_diff">+38</td><td class="right " data-stat="points">76</td><td class="right " data-stat="points_avg">2.00</td><td class="right " data-stat="xg_for">64.6</td><td class="right " data-stat="xg_against">33.4</td><td class="right " data-stat="xg_diff">+31.2</td><td class="right " data-stat="xg_diff_per90">+0.82</td><td class="right " data-stat="attendance_per_g" csk="60883">60,883</td><td class="right " data-stat="top_team_scorers" csk="20"><a href="/en/players/e92cd3f7/Alexander-Sorloth">Alexander Sørloth</a> - <span>20</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/ee8508c0/Jan-Oblak">Jan Oblak</a></td><td class="left " data-stat="notes">→ Champions League via league finish</td></tr> <tr data-row="3"><th scope="row" class="right qualifier qualification_indicator1" data-stat="rank" csk="4">4</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.2b390eca.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/2b390eca/Athletic-Club-Stats">Athletic Club</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">19</td><td class="right " data-stat="ties">13</td><td class="right " data-stat="losses">6</td><td class="right " data-stat="goals_for">54</td><td class="right " data-stat="goals_against">29</td><td class="right " data-stat="goal_diff">+25</td><td class="right " data-stat="points">70</td><td class="right " data-stat="points_avg">1.84</td><td class="right " data-stat="xg_for">53.0</td><td class="right " data-stat="xg_against">37.8</td><td class="right " data-stat="xg_diff">+15.2</td><td class="right " data-stat="xg_diff_per90">+0.40</td><td class="right " data-stat="attendance_per_g" csk="48420">48,420</td><td class="right " data-stat="top_team_scorers" csk="15"><a href="/en/players/bdb624a3/Oihan-Sancet">Oihan Sancet</a> - <span>15</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/5dcf3e90/Unai-Simon">Unai Simón</a></td><td class="left " data-stat="notes">→ Champions League via league finish</td></tr> <tr data-row="4"><th scope="row" class="right qualifier qualification_indicator2" data-stat="rank" csk="5">5</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.2a8183b3.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/2a8183b3/Villarreal-Stats">Villarreal</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">20</td><td class="right " data-stat="ties">10</td><td class="right " data-stat="losses">8</td><td class="right " data-stat="goals_for">71</td><td class="right " data-stat="goals_against">51</td><td class="right " data-stat="goal_diff">+20</td><td class="right " data-stat="points">70</td><td class="right " data-stat="points_avg">1.84</td><td class="right " data-stat="xg_for">64.8</td><td class="right " data-stat="xg_against">44.4</td><td class="right " data-stat="xg_diff">+20.4</td><td class="right " data-stat="xg_diff_per90">+0.54</td><td class="right " data-stat="attendance_per_g" csk="18266">18,266</td><td class="right " data-stat="top_team_scorers" csk="19"><a href="/en/players/819aa8e7/Ayoze-Perez">Ayoze Pérez</a> - <span>19</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/4eae5bc4/Diego-Conde">Diego Conde</a></td><td class="left " data-stat="notes">→ Europa League via league finish</td></tr> <tr data-row="5"><th scope="row" class="right qualifier qualification_indicator2" data-stat="rank" csk="6">6</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.fc536746.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/fc536746/Real-Betis-Stats">Betis</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">16</td><td class="right " data-stat="ties">12</td><td class="right " data-stat="losses">10</td><td class="right " data-stat="goals_for">57</td><td class="right " data-stat="goals_against">50</td><td class="right " data-stat="goal_diff">+7</td><td class="right " data-stat="points">60</td><td class="right " data-stat="points_avg">1.58</td><td class="right " data-stat="xg_for">54.7</td><td class="right " data-stat="xg_against">50.6</td><td class="right " data-stat="xg_diff">+4.1</td><td class="right " data-stat="xg_diff_per90">+0.11</td><td class="right " data-stat="attendance_per_g" csk="51542">51,542</td><td class="right " data-stat="top_team_scorers" csk="9"><a href="/en/players/a0b4bb3e/Isco">Isco</a> - <span>9</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/f76e6b4e/Adrian">Adrián</a></td><td class="left " data-stat="notes">→ Europa League via league finish</td></tr> <tr data-row="6"><th scope="row" class="right qualifier qualification_indicator3" data-stat="rank" csk="7">7</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.f25da7fb.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/f25da7fb/Celta-Vigo-Stats">Celta Vigo</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">16</td><td class="right " data-stat="ties">7</td><td class="right " data-stat="losses">15</td><td class="right " data-stat="goals_for">59</td><td class="right " data-stat="goals_against">57</td><td class="right " data-stat="goal_diff">+2</td><td class="right " data-stat="points">55</td><td class="right " data-stat="points_avg">1.45</td><td class="right " data-stat="xg_for">54.2</td><td class="right " data-stat="xg_against">43.4</td><td class="right " data-stat="xg_diff">+10.9</td><td class="right " data-stat="xg_diff_per90">+0.29</td><td class="right " data-stat="attendance_per_g" csk="21504">21,504</td><td class="right " data-stat="top_team_scorers" csk="11"><a href="/en/players/75645f0e/Borja-Iglesias">Borja Iglesias</a> - <span>11</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/ab13a5aa/Vicente-Guaita">Vicente Guaita</a></td><td class="left " data-stat="notes">→ Conference League via league finish</td></tr> <tr data-row="7"><th scope="row" class="right " data-stat="rank" csk="8">8</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.98e8af82.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/98e8af82/Rayo-Vallecano-Stats">Rayo Vallecano</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">13</td><td class="right " data-stat="ties">13</td><td class="right " data-stat="losses">12</td><td class="right " data-stat="goals_for">41</td><td class="right " data-stat="goals_against">45</td><td class="right " data-stat="goal_diff">-4</td><td class="right " data-stat="points">52</td><td class="right " data-stat="points_avg">1.37</td><td class="right " data-stat="xg_for">45.4</td><td class="right " data-stat="xg_against">49.0</td><td class="right " data-stat="xg_diff">-3.5</td><td class="right " data-stat="xg_diff_per90">-0.09</td><td class="right " data-stat="attendance_per_g" csk="12908">12,908</td><td class="right " data-stat="top_team_scorers" csk="6"><a href="/en/players/38ce1150/Jorge-de-Frutos">Jorge de Frutos</a> - <span>6</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/fc627681/Augusto-Batalla">Augusto Batalla</a></td><td class="left iz" data-stat="notes"></td></tr> <tr data-row="8"><th scope="row" class="right " data-stat="rank" csk="9">9</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.03c57e2b.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/03c57e2b/Osasuna-Stats">Osasuna</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">12</td><td class="right " data-stat="ties">16</td><td class="right " data-stat="losses">10</td><td class="right " data-stat="goals_for">48</td><td class="right " data-stat="goals_against">52</td><td class="right " data-stat="goal_diff">-4</td><td class="right " data-stat="points">52</td><td class="right " data-stat="points_avg">1.37</td><td class="right " data-stat="xg_for">44.0</td><td class="right " data-stat="xg_against">53.7</td><td class="right " data-stat="xg_diff">-9.6</td><td class="right " data-stat="xg_diff_per90">-0.25</td><td class="right " data-stat="attendance_per_g" csk="20476">20,476</td><td class="right " data-stat="top_team_scorers" csk="21"><a href="/en/players/8f3565b3/Ante-Budimir">Ante Budimir</a> - <span>21</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/a4995f48/Sergio-Herrera">Sergio Herrera</a></td><td class="left iz" data-stat="notes"></td></tr> <tr data-row="9"><th scope="row" class="right " data-stat="rank" csk="10">10</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.2aa12281.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/2aa12281/Mallorca-Stats">Mallorca</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">13</td><td class="right " data-stat="ties">9</td><td class="right " data-stat="losses">16</td><td class="right " data-stat="goals_for">35</td><td class="right " data-stat="goals_against">44</td><td class="right " data-stat="goal_diff">-9</td><td class="right " data-stat="points">48</td><td class="right " data-stat="points_avg">1.26</td><td class="right " data-stat="xg_for">38.8</td><td class="right " data-stat="xg_against">46.8</td><td class="right " data-stat="xg_diff">-8.1</td><td class="right " data-stat="xg_diff_per90">-0.21</td><td class="right " data-stat="attendance_per_g" csk="18502">18,502</td><td class="right " data-stat="top_team_scorers" csk="7"><a href="/en/players/e7af9060/Cyle-Larin">Cyle Larin</a>, <a href="/en/players/0cadc46d/Vedat-Muriqi">Vedat Muriqi</a> - <span>7</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/30061f53/Dominik-Greif">Dominik Greif</a></td><td class="left iz" data-stat="notes"></td></tr> <tr data-row="10"><th scope="row" class="right " data-stat="rank" csk="11">11</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.e31d1cd9.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/e31d1cd9/Real-Sociedad-Stats">Real Sociedad</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">13</td><td class="right " data-stat="ties">7</td><td class="right " data-stat="losses">18</td><td class="right " data-stat="goals_for">35</td><td class="right " data-stat="goals_against">46</td><td class="right " data-stat="goal_diff">-11</td><td class="right " data-stat="points">46</td><td class="right " data-stat="points_avg">1.21</td><td class="right " data-stat="xg_for">42.5</td><td class="right " data-stat="xg_against">43.9</td><td class="right " data-stat="xg_diff">-1.5</td><td class="right " data-stat="xg_diff_per90">-0.04</td><td class="right " data-stat="attendance_per_g" csk="29877">29,877</td><td class="right " data-stat="top_team_scorers" csk="9"><a href="/en/players/8c3c640c/Mikel-Oyarzabal">Mikel Oyarzabal</a> - <span>9</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/374ee424/Alex-Remiro">Álex Remiro</a></td><td class="left iz" data-stat="notes"></td></tr> <tr data-row="11"><th scope="row" class="right " data-stat="rank" csk="12">12</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.dcc91a7b.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/dcc91a7b/Valencia-Stats">Valencia</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">11</td><td class="right " data-stat="ties">13</td><td class="right " data-stat="losses">14</td><td class="right " data-stat="goals_for">44</td><td class="right " data-stat="goals_against">54</td><td class="right " data-stat="goal_diff">-10</td><td class="right " data-stat="points">46</td><td class="right " data-stat="points_avg">1.21</td><td class="right " data-stat="xg_for">43.2</td><td class="right " data-stat="xg_against">51.9</td><td class="right " data-stat="xg_diff">-8.6</td><td class="right " data-stat="xg_diff_per90">-0.23</td><td class="right " data-stat="attendance_per_g" csk="43042">43,042</td><td class="right " data-stat="top_team_scorers" csk="11"><a href="/en/players/37e103f8/Hugo-Duro">Hugo Duro</a> - <span>11</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/d4b73232/Giorgi-Mamardashvili">Giorgi Mamardashvili</a></td><td class="left iz" data-stat="notes"></td></tr> <tr data-row="12"><th scope="row" class="right " data-stat="rank" csk="13">13</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.7848bd64.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/7848bd64/Getafe-Stats">Getafe</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">11</td><td class="right " data-stat="ties">9</td><td class="right " data-stat="losses">18</td><td class="right " data-stat="goals_for">34</td><td class="right " data-stat="goals_against">39</td><td class="right " data-stat="goal_diff">-5</td><td class="right " data-stat="points">42</td><td class="right " data-stat="points_avg">1.11</td><td class="right " data-stat="xg_for">36.8</td><td class="right " data-stat="xg_against">46.2</td><td class="right " data-stat="xg_diff">-9.4</td><td class="right " data-stat="xg_diff_per90">-0.25</td><td class="right " data-stat="attendance_per_g" csk="11469">11,469</td><td class="right " data-stat="top_team_scorers" csk="10"><a href="/en/players/58afe001/Mauro-Arambarri">Mauro Arambarri</a> - <span>10</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/8f8c6934/David-Soria">David Soria</a></td><td class="left iz" data-stat="notes"></td></tr> <tr data-row="13"><th scope="row" class="right " data-stat="rank" csk="14">14</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.a8661628.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/a8661628/Espanyol-Stats">Espanyol</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">11</td><td class="right " data-stat="ties">9</td><td class="right " data-stat="losses">18</td><td class="right " data-stat="goals_for">40</td><td class="right " data-stat="goals_against">51</td><td class="right " data-stat="goal_diff">-11</td><td class="right " data-stat="points">42</td><td class="right " data-stat="points_avg">1.11</td><td class="right " data-stat="xg_for">34.4</td><td class="right " data-stat="xg_against">53.8</td><td class="right " data-stat="xg_diff">-19.4</td><td class="right " data-stat="xg_diff_per90">-0.51</td><td class="right " data-stat="attendance_per_g" csk="25640">25,640</td><td class="right " data-stat="top_team_scorers" csk="12"><a href="/en/players/2683eda7/Javi-Puado">Javi Puado</a> - <span>12</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/87b498b0/Joan-Garcia">Joan García</a></td><td class="left iz" data-stat="notes"></td></tr> <tr data-row="14"><th scope="row" class="right " data-stat="rank" csk="15">15</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.8d6fd021.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/8d6fd021/Alaves-Stats">Alavés</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">10</td><td class="right " data-stat="ties">12</td><td class="right " data-stat="losses">16</td><td class="right " data-stat="goals_for">38</td><td class="right " data-stat="goals_against">48</td><td class="right " data-stat="goal_diff">-10</td><td class="right " data-stat="points">42</td><td class="right " data-stat="points_avg">1.11</td><td class="right " data-stat="xg_for">42.8</td><td class="right " data-stat="xg_against">47.0</td><td class="right " data-stat="xg_diff">-4.2</td><td class="right " data-stat="xg_diff_per90">-0.11</td><td class="right " data-stat="attendance_per_g" csk="17318">17,318</td><td class="right " data-stat="top_team_scorers" csk="13"><a href="/en/players/e897d8ba/Kike">Kiké</a> - <span>13</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/9970da54/Antonio-Sivera">Antonio Sivera</a></td><td class="left iz" data-stat="notes"></td></tr> <tr data-row="15"><th scope="row" class="right " data-stat="rank" csk="16">16</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.9024a00a.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/9024a00a/Girona-Stats">Girona</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">11</td><td class="right " data-stat="ties">8</td><td class="right " data-stat="losses">19</td><td class="right " data-stat="goals_for">44</td><td class="right " data-stat="goals_against">60</td><td class="right " data-stat="goal_diff">-16</td><td class="right " data-stat="points">41</td><td class="right " data-stat="points_avg">1.08</td><td class="right " data-stat="xg_for">42.5</td><td class="right " data-stat="xg_against">50.4</td><td class="right " data-stat="xg_diff">-8.0</td><td class="right " data-stat="xg_diff_per90">-0.21</td><td class="right " data-stat="attendance_per_g" csk="11657">11,657</td><td class="right " data-stat="top_team_scorers" csk="11"><a href="/en/players/220c11f1/Cristhian-Stuani">Cristhian Stuani</a> - <span>11</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/63d17038/Paulo-Gazzaniga">Paulo Gazzaniga</a></td><td class="left iz" data-stat="notes"></td></tr> <tr data-row="16"><th scope="row" class="right " data-stat="rank" csk="17">17</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.ad2be733.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/ad2be733/Sevilla-Stats">Sevilla</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">10</td><td class="right " data-stat="ties">11</td><td class="right " data-stat="losses">17</td><td class="right " data-stat="goals_for">42</td><td class="right " data-stat="goals_against">55</td><td class="right " data-stat="goal_diff">-13</td><td class="right " data-stat="points">41</td><td class="right " data-stat="points_avg">1.08</td><td class="right " data-stat="xg_for">42.7</td><td class="right " data-stat="xg_against">47.4</td><td class="right " data-stat="xg_diff">-4.7</td><td class="right " data-stat="xg_diff_per90">-0.12</td><td class="right " data-stat="attendance_per_g" csk="35619">35,619</td><td class="right " data-stat="top_team_scorers" csk="11"><a href="/en/players/0c61c77c/Dodi-Lukebakio">Dodi Lukebakio</a> - <span>11</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/bdedffac/Orjan-Nyland">Ørjan Nyland</a></td><td class="left iz" data-stat="notes"></td></tr> <tr data-row="17"><th scope="row" class="right relegate qualification_indicator" data-stat="rank" csk="18">18</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.7c6f2c78.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/7c6f2c78/Leganes-Stats">Leganés</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">9</td><td class="right " data-stat="ties">13</td><td class="right " data-stat="losses">16</td><td class="right " data-stat="goals_for">39</td><td class="right " data-stat="goals_against">56</td><td class="right " data-stat="goal_diff">-17</td><td class="right " data-stat="points">40</td><td class="right " data-stat="points_avg">1.05</td><td class="right " data-stat="xg_for">36.0</td><td class="right " data-stat="xg_against">59.4</td><td class="right " data-stat="xg_diff">-23.4</td><td class="right " data-stat="xg_diff_per90">-0.62</td><td class="right " data-stat="attendance_per_g" csk="11135">11,135</td><td class="right " data-stat="top_team_scorers" csk="8"><a href="/en/players/ea6a7ab4/Dani-Raba">Dani Raba</a> - <span>8</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/ac295d9e/Marko-Dmitrovic">Marko Dmitrović</a></td><td class="left " data-stat="notes">Relegated</td></tr> <tr data-row="18"><th scope="row" class="right relegate qualification_indicator" data-stat="rank" csk="19">19</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.0049d422.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/0049d422/Las-Palmas-Stats">Las Palmas</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">8</td><td class="right " data-stat="ties">8</td><td class="right " data-stat="losses">22</td><td class="right " data-stat="goals_for">40</td><td class="right " data-stat="goals_against">61</td><td class="right " data-stat="goal_diff">-21</td><td class="right " data-stat="points">32</td><td class="right " data-stat="points_avg">0.84</td><td class="right " data-stat="xg_for">36.3</td><td class="right " data-stat="xg_against">66.8</td><td class="right " data-stat="xg_diff">-30.5</td><td class="right " data-stat="xg_diff_per90">-0.80</td><td class="right " data-stat="attendance_per_g" csk="23010">23,010</td><td class="right " data-stat="top_team_scorers" csk="10"><a href="/en/players/e7aa9d7c/Fabio-Silva">Fábio Silva</a> - <span>10</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/04ac8285/Jasper-Cillessen">Jasper Cillessen</a></td><td class="left " data-stat="notes">Relegated</td></tr> <tr data-row="19"><th scope="row" class="right relegate qualification_indicator" data-stat="rank" csk="20">20</th><td class="left " data-stat="team"><img itemscope="image" height="13" width="13" src="https://cdn.ssref.net/req/202505152/tlogo/fb/mini.17859612.png" alt="Club Crest" style="vertical-align:text-top"> <a href="/en/squads/17859612/Valladolid-Stats">Valladolid</a></td><td class="right " data-stat="games">38</td><td class="right " data-stat="wins">4</td><td class="right " data-stat="ties">4</td><td class="right " data-stat="losses">30</td><td class="right " data-stat="goals_for">26</td><td class="right " data-stat="goals_against">90</td><td class="right " data-stat="goal_diff">-64</td><td class="right " data-stat="points">16</td><td class="right " data-stat="points_avg">0.42</td><td class="right " data-stat="xg_for">34.6</td><td class="right " data-stat="xg_against">67.6</td><td class="right " data-stat="xg_diff">-33.0</td><td class="right " data-stat="xg_diff_per90">-0.87</td><td class="right " data-stat="attendance_per_g" csk="19831">19,831</td><td class="right " data-stat="top_team_scorers" csk="5"><a href="/en/players/e5ec95f8/Mamadou-Sylla">Mamadou Sylla</a> - <span>5</span></td><td class="right " data-stat="top_keeper"><a href="/en/players/aa81d8f8/Karl-Jakob-Hein">Karl Jakob Hein</a></td><td class="left " data-stat="notes">Relegated</td></tr> </tbody> </table>